In [1]:
# importa as bibliotecas

import numpy as np
import pandas as pd
import sqlite3 as sq


In [2]:
# carrega os dados
covid = pd.read_csv("./dados/MICRODADOS.csv", sep=";", encoding="cp1252")

/tmp/ipykernel_2681677/678062200.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  covid = pd.read_csv("./dados/MICRODADOS.csv", sep=";", encoding="cp1252")


In [3]:
# visualiza as colunas
covid.columns

Index(['DataNotificacao', 'DataCadastro', 'DataDiagnostico',
       'DataColeta_RT_PCR', 'DataColetaTesteRapido', 'DataColetaSorologia',
       'DataColetaSorologiaIGG', 'DataEncerramento', 'DataObito',
       'Classificacao', 'Evolucao', 'CriterioConfirmacao', 'StatusNotificacao',
       'Municipio', 'Bairro', 'FaixaEtaria', 'IdadeNaDataNotificacao', 'Sexo',
       'RacaCor', 'Escolaridade', 'Gestante', 'Febre',
       'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia',
       'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio',
       'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo',
       'ComorbidadeObesidade', 'FicouInternado', 'ViagemBrasil',
       'ViagemInternacional', 'ProfissionalSaude', 'PossuiDeficiencia',
       'MoradorDeRua', 'ResultadoRT_PCR', 'ResultadoTesteRapido',
       'ResultadoSorologia', 'ResultadoSorologia_IGG', 'TipoTesteRapido'],
      dtype='object')

In [4]:
# remove quando não houve morte
mortes = covid.dropna(subset=['DataObito']).reset_index(drop=True)

# dataframe para a dimensão municipio
municipioDim = pd.DataFrame(mortes['Municipio'].unique(), columns=["Municipio"])
# dataframe para a dimensão sexo
sexoDim = pd.DataFrame(mortes['Sexo'].unique(), columns=["Sexo"])
# dataframe para a dimensão viagem internacional
viagemIDim = pd.DataFrame(mortes['ViagemInternacional'].unique(), columns=["ViagemInternacional"])
# dataframe para a dimensão data
dataDim = pd.DataFrame(mortes['DataObito'].unique(), columns=["DataObito"])



In [5]:
# configuração do banco sqlite
conn = sq.connect('mortes.db')

# cursor para fazer operações no banco
cur = conn.cursor()

# dropa as tabelas se existirem
cur.execute("PRAGMA foreign_keys = ON;")
cur.execute("DROP TABLE IF EXISTS Obito_Fato")
cur.execute("DROP TABLE IF EXISTS Municipio_Dim")
cur.execute("DROP TABLE IF EXISTS Sexo_Dim")
cur.execute("DROP TABLE IF EXISTS Viagem_Internacional_Dim")
cur.execute("DROP TABLE IF EXISTS Data_Obito_Dim")

# cria as tabelas
cur.execute("""
CREATE TABLE IF NOT EXISTS Municipio_Dim (
	municipio_id INTEGER PRIMARY KEY AUTOINCREMENT,
	municipio VARCHAR(100) NOT NULL
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Sexo_Dim (
	sexo_id INTEGER PRIMARY KEY AUTOINCREMENT,
   	sexo CHARACTER(1) NOT NULL
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Viagem_Internacional_Dim (
	viagem_internacional_id INTEGER PRIMARY KEY AUTOINCREMENT,
   	viagem_internacional_dim VARCHAR(40) NOT NULL
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Data_Obito_Dim (
	data_obito_id INTEGER PRIMARY KEY AUTOINCREMENT,
   	data_obito DATE NOT NULL
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Obito_Fato (
	obito_id INTEGER PRIMARY KEY AUTOINCREMENT,
    municipio_id_fk INTEGER,
	sexo_id_fk INTEGER,
	viagem_internacional_id_fk INTEGER,
	data_obito_id_fk INTEGER,
    FOREIGN KEY(municipio_id_fk) REFERENCES Municipio_Dim(municipio_id),
    FOREIGN KEY(sexo_id_fk) REFERENCES Sexo_Dim(sexo_id),
    FOREIGN KEY(viagem_internacional_id_fk) REFERENCES Viagem_Internacional_Dim(viagem_internacional_id),
    FOREIGN KEY(data_obito_id_fk) REFERENCES Data_Obito_Dim(data_obito_id)
);
""")

In [6]:
# seleciona tudo da tabela Obito_Fato para teste
cur.execute("SELECT * FROM Obito_Fato").fetchall()

[]

In [7]:
cur.executemany("INSERT INTO Municipio_Dim (municipio) VALUES(?)", municipioDim.values)
conn.commit()
cur.execute("SELECT * FROM Municipio_Dim").fetchall()

[(1, 'CACHOEIRO DE ITAPEMIRIM'),
 (2, 'GUARAPARI'),
 (3, 'PRESIDENTE KENNEDY'),
 (4, 'LINHARES'),
 (5, 'SERRA'),
 (6, 'AGUA DOCE DO NORTE'),
 (7, 'VILA VELHA'),
 (8, 'CASTELO'),
 (9, 'CARIACICA'),
 (10, 'PEDRO CANARIO'),
 (11, 'VITORIA'),
 (12, 'COLATINA'),
 (13, 'CONCEICAO DA BARRA'),
 (14, 'ARACRUZ'),
 (15, 'DOMINGOS MARTINS'),
 (16, 'MONTANHA'),
 (17, 'BARRA DE SAO FRANCISCO'),
 (18, 'SAO MATEUS'),
 (19, 'ITAPEMIRIM'),
 (20, 'ANCHIETA'),
 (21, 'VIANA'),
 (22, 'PIUMA'),
 (23, 'IBIRACU'),
 (24, 'AFONSO CLAUDIO'),
 (25, 'MARATAIZES'),
 (26, 'MANTENOPOLIS'),
 (27, 'SAO GABRIEL DA PALHA'),
 (28, 'ALEGRE'),
 (29, 'JOAO NEIVA'),
 (30, 'ITAGUACU'),
 (31, 'BOA ESPERANCA'),
 (32, 'SOORETAMA'),
 (33, 'ECOPORANGA'),
 (34, 'ALFREDO CHAVES'),
 (35, 'PINHEIROS'),
 (36, 'URUCUCA'),
 (37, 'JERONIMO MONTEIRO'),
 (38, 'NOVA VENECIA'),
 (39, 'SANTA TERESA'),
 (40, 'VENDA NOVA DO IMIGRANTE'),
 (41, 'ITARANA'),
 (42, 'FUNDAO'),
 (43, 'SANTA LEOPOLDINA'),
 (44, 'MIMOSO DO SUL'),
 (45, 'MUQUI'),
 (46, 'GOV

In [8]:
cur.executemany("INSERT INTO Sexo_Dim (sexo) VALUES(?)", sexoDim.values)
conn.commit()
cur.execute("SELECT * FROM Sexo_Dim").fetchall()

[(1, 'F'), (2, 'M'), (3, 'I')]

In [9]:
# [item for item in viagemIDim['ViagemInternacional']]

cur.executemany("INSERT INTO Viagem_Internacional_Dim (viagem_internacional_dim) VALUES(?)", viagemIDim.values)
conn.commit()
cur.execute("SELECT * FROM Viagem_Internacional_Dim").fetchall()

[(1, 'Não'), (2, 'Ignorado'), (3, 'Não Informado'), (4, 'Sim')]

In [10]:
cur.executemany("INSERT INTO Data_Obito_Dim (data_obito) VALUES(?)", dataDim.values)
conn.commit()
cur.execute("SELECT * FROM Data_Obito_Dim").fetchall()

[(1, '2023-04-13'),
 (2, '2023-04-17'),
 (3, '2023-04-15'),
 (4, '2023-04-14'),
 (5, '2023-04-10'),
 (6, '2023-04-12'),
 (7, '2023-04-03'),
 (8, '2023-04-11'),
 (9, '2023-04-09'),
 (10, '2023-04-16'),
 (11, '2023-04-07'),
 (12, '2023-04-19'),
 (13, '2023-04-05'),
 (14, '2023-04-04'),
 (15, '2023-03-31'),
 (16, '2023-03-30'),
 (17, '2023-03-27'),
 (18, '2023-04-18'),
 (19, '2023-04-01'),
 (20, '2023-03-29'),
 (21, '2023-04-08'),
 (22, '2023-03-26'),
 (23, '2023-04-02'),
 (24, '2023-03-17'),
 (25, '2023-03-28'),
 (26, '2023-03-23'),
 (27, '2023-03-18'),
 (28, '2023-03-22'),
 (29, '2023-03-21'),
 (30, '2023-03-20'),
 (31, '2023-03-15'),
 (32, '2023-03-16'),
 (33, '2023-03-19'),
 (34, '2023-03-13'),
 (35, '2023-03-14'),
 (36, '2022-12-23'),
 (37, '2023-03-05'),
 (38, '2023-03-11'),
 (39, '2023-03-12'),
 (40, '2023-03-09'),
 (41, '2023-03-08'),
 (42, '2023-03-10'),
 (43, '2023-03-07'),
 (44, '2023-03-06'),
 (45, '2023-03-03'),
 (46, '2023-03-02'),
 (47, '2023-02-28'),
 (48, '2023-02-06'),
 

In [11]:
lista_obitos = []

for morte in mortes.get(['Municipio', 'Sexo', 'ViagemInternacional', 'DataObito']).itertuples():
    lista_obitos.append((
        int(municipioDim.index[municipioDim['Municipio'] == morte[1]][0]) + 1,
        int(sexoDim.index[sexoDim['Sexo'] == morte[2]][0]) + 1,
        int(viagemIDim.index[viagemIDim['ViagemInternacional'] == morte[3]][0]) + 1,
        int(dataDim.index[dataDim['DataObito'] == morte[4]][0]) + 1,
    ))

cur.executemany("INSERT INTO Obito_Fato (municipio_id_fk, sexo_id_fk, viagem_internacional_id_fk, data_obito_id_fk) VALUES(?, ?, ?, ?)", lista_obitos)
conn.commit()
cur.execute("SELECT * FROM Obito_Fato").fetchmany(20)

[(1, 1, 1, 1, 1),
 (2, 2, 1, 1, 2),
 (3, 3, 2, 1, 3),
 (4, 4, 1, 1, 3),
 (5, 5, 1, 2, 4),
 (6, 6, 2, 1, 5),
 (7, 7, 1, 1, 4),
 (8, 8, 1, 1, 6),
 (9, 9, 2, 1, 7),
 (10, 10, 2, 1, 4),
 (11, 1, 2, 1, 1),
 (12, 11, 2, 1, 8),
 (13, 12, 1, 1, 8),
 (14, 5, 1, 1, 5),
 (15, 5, 1, 1, 6),
 (16, 13, 1, 1, 5),
 (17, 14, 2, 1, 9),
 (18, 6, 1, 1, 9),
 (19, 9, 2, 1, 10),
 (20, 5, 2, 1, 11)]